In [1]:
import zipfile
import os
import sys
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import roc_curve
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [2]:
TRAIN_PATH= 'training'
TEST_PATH= 'testing'

datagen = ImageDataGenerator(rescale=1.0/255.0, featurewise_center=True)

datagen.mean = [123.68, 116.779, 103.939]

train_gen = datagen.flow_from_directory(TRAIN_PATH, class_mode='binary', 
                                      batch_size=64, target_size=(224,224))
test_gen = datagen.flow_from_directory(TEST_PATH, class_mode='binary', 
                                      batch_size=64, target_size=(224,224))

Found 18750 images belonging to 2 classes.
Found 6250 images belonging to 2 classes.


In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform',
                    padding='same', input_shape=(224,224,3)))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    opt=SGD(lr=0.01, momentum=0.9)
    
    model.compile(optimizer=opt, metrics=['acc',f1_m,precision_m, recall_m], loss='binary_crossentropy')
    return model

In [5]:
model = define_model()

rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                              min_delta=1E-7)

history = model.fit_generator(train_gen, steps_per_epoch=len(train_gen), callbacks=[rlrop],
         epochs=10, validation_data=test_gen, verbose=1)

/Users/basma.elsaify/.pyenv/versions/3.8.3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


293/293 [==============================] - 203s 689ms/step - loss: 1.2323 - acc: 0.5812 - f1_m: 0.5380 - precision_m: 0.5956 - recall_m: 0.5920 - val_loss: 0.5876 - val_acc: 0.6822 - val_f1_m: 0.6339 - val_precision_m: 0.7454 - val_recall_m: 0.5578


In [6]:
history.history

{'loss': [0.7494252324104309],
 'acc': [0.6262933611869812],
 'f1_m': [0.593091607093811],
 'precision_m': [0.6603574752807617],
 'recall_m': [0.6043522953987122],
 'val_loss': [0.5875988006591797],
 'val_acc': [0.6822400093078613],
 'val_f1_m': [0.6338533759117126],
 'val_precision_m': [0.7453943490982056],
 'val_recall_m': [0.5577754378318787],
 'lr': [0.01]}

In [20]:
def summarize_diagnostics(history):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['acc'], color='blue', label='train')
    plt.plot(history.history['val_acc'], color='orange', label='test')
    plt.show()
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

summarize_diagnostics(history)

In [12]:
evaluated = model.evaluate_generator(test_gen,  steps=len(test_gen), verbose=1)
print(evaluated)

[0.5875987410545349, 0.6822400093078613, 0.633675754070282, 0.7449535727500916, 0.5574128031730652]


In [13]:
def define_model():
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    
    for layer in model.layers:
        layer.trainable = False
    
    flat = Flatten()(model.layers[-1].output)
    dense1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat)
    output = Dense(1, activation='sigmoid')(dense1)
    
    model = Model(inputs=model.inputs, outputs=output)
    
    
    # Learning rate has been reduced to 0.001 as the model is already trained
    opt=SGD(lr=0.001, momentum=0.9)
    
    model.compile(optimizer=opt, metrics=['acc',f1_m,precision_m, recall_m], loss='binary_crossentropy')
    return model

In [15]:
model = define_model()

history = model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=10, 
                              validation_data=test_gen, validation_steps=len(test_gen), 
                              verbose=1)

293/293 [==============================] - 1704s 6s/step - loss: 1.0476 - accuracy: 0.9291 - val_loss: 0.0709 - val_accuracy: 0.9723


In [17]:
summarize_diagnostics(history)

In [19]:
evaluated = model.evaluate_generator(test_gen,  steps=len(test_gen), verbose=1)
print(evaluated)

98/98 [==============================] - 403s 4s/step - loss: 0.0709 - accuracy: 0.9723
[0.07094848155975342, 0.972320020198822]
